## ThinkDSP

This notebook contains code examples from Chapter 4: Noise

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import thinkdsp
import thinkplot
import thinkstats2 

import numpy as np

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [ ]:
thinkdsp.random_seed(17)

The simplest noise to generate is uncorrelated uniform (UU) noise:

In [ ]:
signal = thinkdsp.UncorrelatedUniformNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
wave.make_audio()

Here's what a segment of it looks like:

In [ ]:
segment = wave.segment(duration=0.1)
segment.plot(linewidth=1)
thinkplot.config(xlabel='time',
                 ylabel='amplitude',
                 ylim=[-1.05, 1.05],
                 legend=False)

And here's the spectrum:

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='amplitude',
                 xlim=[0, spectrum.fs[-1]])

In the context of noise it is more conventional to look at the spectrum of power, which is the square of amplitude:

In [ ]:
spectrum.plot_power(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xlim=[0, spectrum.fs[-1]])

UU noise has the same power at all frequencies, on average, which we can confirm by looking at the normalized cumulative sum of power, which I call an integrated spectrum:

In [ ]:
integ = spectrum.make_integrated_spectrum()
integ.plot_power()
thinkplot.config(xlabel='frequency (Hz)',
                ylabel='cumulative power',
                xlim=[0, spectrum.fs[-1]])

A straight line in this figure indicates that UU noise has equal power at all frequencies, on average.  By analogy with light, noise with this property is called "white noise".

### Brownian noise

Brownian noise is generated by adding up a sequence of random steps.

In [ ]:
signal = thinkdsp.BrownianNoise()
wave = signal.make_wave(duration=5, framerate=11025)
wave.make_audio()

The sound is less bright, or more muffled, than white noise.

Here's what the wave looks like:

In [ ]:
wave.plot(linewidth=1)
thinkplot.config(xlabel='time',
                 ylabel='amplitude',
                 ylim=[-1.05, 1.05])

Here's what the power spectrum looks like on a linear scale.

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xlim=[0, spectrum.fs[-1]])

So much of the energy is at low frequencies, we can't even see the high frequencies.

We can get a better view by plotting the power spectrum on a log-log scale.

In [ ]:
# The f=0 component is very small, so on a log scale
# it's very negative.  If we clobber it before plotting,
# we can see the rest of the spectrum better.
spectrum.hs[0] = 0

spectrum.plot_power(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xscale='log',
                 yscale='log',
                 xlim=[0, spectrum.fs[-1]])

Now the relationship between power and frequency is clearer.  The slope of this line is approximately -2, which indicates that $P = K / f^2$, for some constant $K$.

In [ ]:
signal = thinkdsp.BrownianNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
spectrum = wave.make_spectrum()
result = spectrum.estimate_slope()
result.slope

The estimated slope of the line is closer to -1.8 than -2, for reasons we'll see later.

### Pink noise

Pink noise is characterized by a parameter, $\beta$, usually between 0 and 2.  You can hear the differences below.

With $\beta=0$, we get white noise:

In [ ]:
signal = thinkdsp.PinkNoise(beta=0)
wave = signal.make_wave(duration=5)
wave.make_audio()

With $\beta=1$, pink noise has the relationship $P = K / f$, which is why it is also called $1/f$ noise.

In [ ]:
signal = thinkdsp.PinkNoise(beta=1)
wave = signal.make_wave(duration=5)
wave.make_audio()

With $\beta=2$, we get Brownian (aka red) noise.

In [ ]:
signal = thinkdsp.PinkNoise(beta=2)
wave = signal.make_wave(duration=5)
wave.make_audio()

The following figure shows the power spectrums for white, pink, and red noise on a log-log scale.

In [ ]:
colors = ['#9ecae1', '#4292c6', '#2171b5']
betas = [0, 1, 2]

for beta, color in zip(betas, colors):
    signal = thinkdsp.PinkNoise(beta=beta)
    wave = signal.make_wave(duration=5, framerate=1024)
    spectrum = wave.make_spectrum()
    spectrum.hs[0] = 0
    spectrum.plot_power(linewidth=1, color=color)
    
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xscale='log',
                 yscale='log',
                 xlim=[0, spectrum.fs[-1]])

### Uncorrelated Gaussian noise

An alternative to UU noise is uncorrelated Gaussian (UG noise).

In [ ]:
signal = thinkdsp.UncorrelatedGaussianNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
wave.plot(linewidth=0.5)
thinkplot.config(xlabel='time',
                 ylabel='amplitude')

The spectrum of UG noise is also UG noise.

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power(linewidth=1)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xlim=[0, spectrum.fs[-1]])

We can use a normal probability plot to test the distribution of the power spectrum.

In [ ]:
from thinkstats2 import NormalProbabilityPlot

NormalProbabilityPlot(spectrum.real, label='real part')
thinkplot.config(xlabel='normal sample',
                 ylabel='power',
                 ylim=[-250, 250],
                 legend=True,
                 loc='lower right')

A straight line on a normal probability plot indicates that the distribution of the real part of the spectrum is Gaussian.

In [ ]:
NormalProbabilityPlot(spectrum.imag, label='imag part')
thinkplot.config(xlabel='normal sample',
                 ylabel='power',
                 ylim=[-250, 250],
                 legend=True,
                 loc='lower right')

And so is the imaginary part.

# Excercises

## 4.1

“A Soft Murmur” is a web site that plays a mixture of natural
noise sources, including rain, waves, wind, etc. At http://asoftmurmur.
com/about/ you can find their list of recordings, most of which are at http:
//freesound.org.

Download a few of these files and compute the spectrum of each signal.
Does the power spectrum look like white noise, pink noise, or Brownian
noise? How does the spectrum vary over time?

Waves

In [ ]:
# Read
waveLS = thinkdsp.read_wave('132736__ciccarelli__ocean-waves.wav')
waveLS.make_audio()

In [ ]:
# Short segment
segmentLS = waveLS.segment(start=2, duration=2.0)
segmentLS.make_audio()

In [ ]:
spectrumLS = segmentLS.make_spectrum()
spectrumLS.plot_power(1500)
thinkplot.config(xlabel='Frequency (Hz)')

In [ ]:
# Power
spectrumLS.plot_power()
thinkplot.config(xlabel='Frequency (Hz)',
                 xscale='log', 
                 yscale='log')

In [ ]:
# Second segment
segmentLS2 = waveLS.segment(start=10, duration=3.0)
segmentLS2.make_audio()

In [ ]:
spectrumLS2 = segmentLS2.make_spectrum()
spectrumLS.plot_power(1500)
spectrumLS2.plot_power(1500, color='#beaed4')
thinkplot.config(xlabel='Frequency (Hz)',
                 ylabel='Amplitude')

In [ ]:
spectrumLS.plot_power(1500)
spectrumLS2.plot_power(1500, color='#beaed4')
thinkplot.config(xlabel='Frequency (Hz)',
                 ylabel='Amplitude',
                 xscale='log', 
                 yscale='log')

In [ ]:
# Spectogram
segmentLS.make_spectrogram(512).plot(high=5000)

In [ ]:
segmentLS2.make_spectrogram(512).plot(high=5000)

Thunder

In [ ]:
# Read
waveTH = thinkdsp.read_wave('237729__flathill__rain-and-thunder-4.wav')
waveTH.make_audio()

In [ ]:
# Short segment
segmentTH = waveTH.segment(start=2, duration=5.0)
segmentTH.make_audio()

In [ ]:
spectrumTH = segmentTH.make_spectrum()
spectrumTH.plot_power(800)
thinkplot.config(xlabel='Frequency (Hz)')

In [ ]:
spectrumTH.plot_power()
thinkplot.config(xlabel='Frequency (Hz)',
                 xscale='log', 
                 yscale='log')

In [ ]:
# Second segment
segmentTH2 = waveTH.segment(start=25, duration=3.0)
segmentTH2.make_audio()

In [ ]:
#Spectrum 
spectrumTH2 = segmentTH2.make_spectrum()
spectrumTH2.plot_power(800)
spectrumTH2.plot_power(800, color='#beaed4')
thinkplot.config(xlabel='Frequency (Hz)',
                 ylabel='Amplitude')

In [ ]:
spectrumTH = segmentTH2.make_spectrum()
spectrumTH2.plot_power(100)

In [ ]:
# Power
spectrumTH.plot_power(1500)
spectrumTH2.plot_power(1500, color='#beaed4')
thinkplot.config(xlabel='Frequency (Hz)',
                 ylabel='Amplitude',
                 xscale='log', 
                 yscale='log')

In [ ]:
# Spectogram
segmentTH.make_spectrogram(512).plot(high=5000)

In [ ]:
# Spectogram
segmentTH2.make_spectrogram(512).plot(high=5000)

## 4.2

In a noise signal, the mixture of frequencies changes over time.
In the long run, we expect the power at all frequencies to be equal, but in
any sample, the power at each frequency is random.

To estimate the long-term average power at each frequency, we can break
a long signal into segments, compute the power spectrum for each segment, and then compute the average across the segments. You can read
more about this algorithm at http://en.wikipedia.org/wiki/Bartlett's_method.

Implement Bartlett’s method and use it to estimate the power spectrum for
a noise wave. Hint: look at the implementation of make_spectrogram.

In [ ]:
def bartlett_method(wave, seg_length=512, win_flag=True):
    """Estimates the power spectrum of a noise wave.
    
    wave: Wave
    seg_length: segment length
    """
    # make a spectrogram and extract the spectrums
    spectro = wave.make_spectrogram(seg_length, win_flag)
    spectrums = spectro.spec_map.values()
    
    # extract the power array from each spectrum
    psds = [spectrum.power for spectrum in spectrums]
    
    # compute the root mean power (which is like an amplitude)
    hs = np.sqrt(sum(psds) / len(psds))
    fs = next(iter(spectrums)).fs
    
    # make a Spectrum with the mean amplitudes
    spectrum = thinkdsp.Spectrum(hs, fs, wave.framerate)
    return spectrum

`bartlett_method` makes a spectrogram and extracts `spec_map`, which maps from times to Spectrum objects.  It computes the PSD for each spectrum, adds them up, and puts the results into a Spectrum object.

In [ ]:
psd = bartlett_method(segmentLS)
psd2 = bartlett_method(segmentLS2)

psd.plot_power()
psd2.plot_power(color='#beaed4')

thinkplot.config(xlabel='Frequency (Hz)', 
                 ylabel='Power', 
                 xscale='log', 
                 yscale='log')

In [ ]:
psd = bartlett_method(segmentFP)
psd2 = bartlett_method(segmentFP2)

psd.plot_power()
psd2.plot_power(color='#beaed4')

thinkplot.config(xlabel='Frequency (Hz)', 
                 ylabel='Power', 
                 xscale='log', 
                 yscale='log')

Now we can see the relationship between power and frequency more clearly.

## 4.3

At http://www.coindesk.com you can download the daily
price of a BitCoin as a CSV file. Read this file and compute the spectrum
of BitCoin prices as a function of time. Does it resemble white, pink, or
Brownian noise?

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('coindesk-bpi-USD-close.csv', nrows=1625, parse_dates=[0])
df.head(10)

In [ ]:
ys =  df.Close.values
ts = np.arange(len(ys))
ts

In [ ]:
wave = thinkdsp.Wave(ys, ts, framerate=1)
wave.plot()
thinkplot.config(xlabel='Time (days)')

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power()
thinkplot.config(xlabel='Frequency (1/days)',
                 xscale='log', yscale='log')

In [ ]:
spectrum.estimate_slope()[0]

Looking a lot like red noise!


## 4.4 

A Geiger counter is a device that detects radiation. When an ionizing particle strikes the detector, it outputs a surge of current. The total output at a point in time can be modeled as uncorrelated Poisson (UP) noise, where each sample is a random quantity from a Poisson distribution, which corresponds to the number of particles detected during an interval.

Write a class called UncorrelatedPoissonNoise that inherits from thinkdsp._Noise and provides evaluate. It should use np.random.poisson to generate random values from a Poisson distribution. The parameter of
this function, lam, is the average number of particles during each interval.
You can use the attribute amp to specify lam. For example, if the framerate is
10 kHz and amp is 0.001, we expect about 10 “clicks” per second.

Generate about a second of UP noise and listen to it. For low values of
amp, like 0.001, it should sound like a Geiger counter. For higher values it
should sound like white noise. Compute and plot the power spectrum to
see whether it looks like white noise.


In [ ]:
class UncorrelatedPoissonNoise(thinkdsp._Noise):
    """Represents uncorrelated Poisson noise."""

    def evaluate(self, ts):
        """Evaluates the signal at the given times.

        ts: float array of times
        
        returns: float wave array
        """
        ys = np.random.poisson(self.amp, len(ts))
        return ys

In [ ]:
amp = 0.001
framerate = 10000
duration = 1

signal = UncorrelatedPoissonNoise(amp=amp)
wave = signal.make_wave(duration=duration, framerate=framerate)
wave.make_audio()

In [ ]:
expected = amp * framerate * duration
actual = sum(wave.ys)
print(expected, actual)

In [ ]:
wave.plot()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power()
thinkplot.config(xlabel='Frequency (Hz)',
                 ylabel='Power',
                 xscale='log', 
                 yscale='log')

In [ ]:
spectrum.estimate_slope().slope

In [ ]:
#Sound more like white noise:
amp = 1
framerate = 10000
duration = 1

signal = UncorrelatedPoissonNoise(amp=amp)
wave = signal.make_wave(duration=duration, framerate=framerate)
wave.make_audio()

In [ ]:
wave.plot()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.hs[0] = 0

thinkplot.preplot(2, cols=2)
thinkstats2.NormalProbabilityPlot(spectrum.real, label='real')
thinkplot.config(xlabel='Normal sample',
                 ylabel='Power',
                 legend=True,
                 loc='lower right')

thinkplot.subplot(2)
thinkstats2.NormalProbabilityPlot(spectrum.imag, label='imag')
thinkplot.config(xlabel='Normal sample',
                     loc='lower right')

## 4.5

The algorithm in this chapter for generating pink noise is conceptually simple but computationally expensive. There are more efficient
alternatives, like the Voss-McCartney algorithm. Research this method, implement it, compute the spectrum of the result, and confirm that it has the
desired relationship between power and frequency.